## Preprocessing (pipeline)

In [50]:
import pandas as pd
import numpy as np
from sklearn import set_config
set_config(transform_output = "pandas")

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from category_encoders import WOEEncoder 
from sklearn.model_selection import GridSearchCV
from skopt import BayesSearchCV

from sklearn.metrics import roc_auc_score, make_scorer

from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline, make_union
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import lightgbm as lgb

BayesSearchCV uses Bayesian optimization techniques to search for the best hyperparameters.
It employs a probabilistic model to approximate the objective function (model performance) and decides the next set of hyperparameters to evaluate based on this approximation.
Unlike GridSearchCV, it does not search through all possible combinations of hyperparameters. Instead, it iteratively selects the most promising set of hyperparameters based on the model's performance observed so far.
Bayesian optimization tends to be more efficient in finding good hyperparameters compared to grid search, especially for high-dimensional or continuous hyperparameter spaces.

In summary, while GridSearchCV performs an exhaustive search over a predefined grid of hyperparameters, BayesSearchCV uses Bayesian optimization to efficiently explore the hyperparameter space and find promising configurations. BayesSearchCV is often preferred when dealing with complex or high-dimensional hyperparameter spaces where an exhaustive search becomes impractical.

#### Load the data

In [51]:
data_train = pd.read_csv('../datasets/train.csv')
data_test = pd.read_csv('../datasets/test.csv')
pd.set_option('display.max_columns', None)

### **missing for now: outlier detection**
### **also look into this encoder for categorical variables: from category_encoders.cat_boost import CatBoostEncoder**

Random Forests, being an ensemble of decision trees, are generally not sensitive to the scale of numeric features. The reason is that decision trees make splits based on feature values but do not rely on the absolute scale of those values. Therefore, in many cases, scaling is not a strict requirement when using Random Forests. --> no standardization for now so we keep interpretability

### Modeling
- we get (1) a labeled dataset (train.csv) and (2) an unlabeled dataset (test.csv)
- split train.csv into a train and test set
- that train set, u should split into train and validation sets (stratified CV split because imbalance)
- that test set has labels, so u can compare the predictions on X_test, y_test with the labels to evaluate performance of the different models **NOTE: to fit a model on the test set that is coming from train.csv, u need to pass the tuned values of the hyperparameters (tuned on the validation set)**
- choose the best performing model 
- then make predictions on test.csv (unlabeled) and export to a csv file which you upload to the website

 note: after finding the optimal parameters, put the values in the pipeline (paramters of RandomForestRegressor)

### Other ideas 
- change objective function? to account for top 20 evaluation metric?
- use proftree? proflogit?

#### Missing values

In [52]:
# For training data
missing_count = data_train.isnull().sum() 
missing_data = pd.DataFrame({'Column Name': missing_count.index, 'Missing Count': missing_count.values})
missing_data = missing_data.sort_values(by='Missing Count', ascending=False)
missing_data = missing_data[missing_data['Missing Count'] > 0]
missing_data

,Column Name,Missing Count
22,Dropped_calls_ratio,4
23,Usage_Band,4
25,call_cost_per_min,4


In [53]:
rows_with_missing_values_train = data_train[data_train.isnull().any(axis=1)]
print("Rows with Missing Values in training data:")
rows_with_missing_values_train

Rows with Missing Values in training data:


,Gender,Age,Connect_Date,L_O_S,Dropped_Calls,tariff,Handset,Peak_calls_Sum,Peak_mins_Sum,OffPeak_calls_Sum,OffPeak_mins_Sum,Weekend_calls_Sum,Weekend_mins_Sum,International_mins_Sum,Nat_call_cost_Sum,AvePeak,AveOffPeak,AveWeekend,National_calls,National mins,AveNational,All_calls_mins,Dropped_calls_ratio,Usage_Band,Mins_charge,call_cost_per_min,actual call cost,Total_call_cost,Total_Cost,Tariff_OK,average cost min,Peak ratio,OffPeak ratio,Weekend ratio,Nat-InterNat Ratio,high Dropped calls,No Usage,target,id
1736,F,48.0,26/07/98,26.966667,2.0,Play 100,BS110,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,-600.0,NaN,0.0,0.0,59.94,OK,0.5,0.0,0.0,0.0,0.0,F,T,0,K244380
3237,F,34.0,22/03/97,43.333333,2.0,Play 100,BS110,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,-600.0,NaN,0.0,0.0,59.94,OK,0.5,0.0,0.0,0.0,0.0,F,T,0,K244320
3836,M,21.0,03/01/96,58.133333,2.0,Play 100,CAS30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,-600.0,NaN,0.0,0.0,59.94,OK,0.5,0.0,0.0,0.0,0.0,F,T,1,K213590
4301,F,22.0,08/08/98,26.533333,5.0,Play 100,CAS30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,-600.0,NaN,0.0,0.0,59.94,OK,0.5,0.0,0.0,0.0,0.0,F,T,1,K212820


In [54]:
# For test data
missing_count = data_test.isnull().sum() 
missing_data = pd.DataFrame({'Column Name': missing_count.index, 'Missing Count': missing_count.values})
missing_data = missing_data.sort_values(by='Missing Count', ascending=False)
missing_data = missing_data[missing_data['Missing Count'] > 0]
missing_data

,Column Name,Missing Count
22,Dropped_calls_ratio,1
23,Usage_Band,1
25,call_cost_per_min,1


In [55]:
rows_with_missing_values_test = data_test[data_test.isnull().any(axis=1)]
print("\nRows with Missing Values in test data:")
rows_with_missing_values_test


Rows with Missing Values in test data:


,Gender,Age,Connect_Date,L_O_S,Dropped_Calls,tariff,Handset,Peak_calls_Sum,Peak_mins_Sum,OffPeak_calls_Sum,OffPeak_mins_Sum,Weekend_calls_Sum,Weekend_mins_Sum,International_mins_Sum,Nat_call_cost_Sum,AvePeak,AveOffPeak,AveWeekend,National_calls,National mins,AveNational,All_calls_mins,Dropped_calls_ratio,Usage_Band,Mins_charge,call_cost_per_min,actual call cost,Total_call_cost,Total_Cost,Tariff_OK,average cost min,Peak ratio,OffPeak ratio,Weekend ratio,Nat-InterNat Ratio,high Dropped calls,No Usage,id
1389,F,34.0,07/09/98,24.858347,2.0,Play 100,BS110,7.0,0.092169,2.0,12.29251,2.0,-3.12627,-3.215572,-5.011147,3.519628,2.912569,0.27729,-22.0,-2.090036,-0.10749,-20.274408,NaN,NaN,-599.241325,NaN,-3.291928,-2.74686,66.563274,OK,0.510543,0.001085,-0.017429,-0.003596,-0.004193,F,T,K689673


**we will impute this since it's so little rows**

In [56]:
target_column = 'target'

# Separate features and target variable
X_train = data_train.drop(target_column, axis=1)
y_train = data_train[target_column]

X_test = data_test

don't know if this should be done after splitting or not -- still need to change this so that test uses the same day 1 as train data

In [57]:
def process_date_column(data, date_column):
    # Convert the date column to datetime format
    data[date_column] = pd.to_datetime(data[date_column], format='%d/%m/%y')

    # Find the earliest date
    earliest_date = data[date_column].min()

    # Convert the date column to days since the earliest date
    data[date_column] = (data[date_column] - earliest_date).dt.days

    return data

X_train = process_date_column(X_train, 'Connect_Date')
X_test = process_date_column(X_test, 'Connect_Date')

#### Check the correlation

#### Split data into train and validation set 

target variable is binary and imbalanced (with the minority class having a frequency of 15%), so using a stratified splitting approach is recommended to ensure that both the training and validation sets have a similar distribution of the target variable.

In [58]:
stratified_splitter = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for train_index, valid_index in stratified_splitter.split(X_train, y_train):
    X_train_split, X_valid_split = X_train.iloc[train_index], X_train.iloc[valid_index]
    y_train_split, y_valid_split = y_train.iloc[train_index], y_train.iloc[valid_index]
    # Now you can use X_train_split, y_train_split for training and X_valid_split, y_valid_split for validation

In [59]:
total_train_samples = X_train_split.shape[0] + X_valid_split.shape[0]
train_distribution_percentage = (X_train_split.shape[0]/ total_train_samples) * 100
validation_distribution_percentage = (X_valid_split.shape[0] / total_train_samples) * 100

print(f"Training Set Distribution: {train_distribution_percentage:.2f}% ({X_train_split.shape[0]} rows)")
print(f"Validation Set Distribution: {validation_distribution_percentage:.2f}% ({X_valid_split.shape[0]} rows)")

Training Set Distribution: 80.02% (4036 rows)
Validation Set Distribution: 19.98% (1008 rows)


#### Pipeline

In [60]:
X_train_split['Tariff_OK'] = np.where(X_train_split['Tariff_OK'] == 'OK', 1, 0)
X_valid_split['Tariff_OK'] = np.where(X_valid_split['Tariff_OK'] == 'OK', 1, 0)
X_test['Tariff_OK'] = np.where(X_test['Tariff_OK'] == 'OK', 1, 0)

C:\Users\lenne\AppData\Local\Temp\ipykernel_2732\328505691.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_split['Tariff_OK'] = np.where(X_train_split['Tariff_OK'] == 'OK', 1, 0)
C:\Users\lenne\AppData\Local\Temp\ipykernel_2732\328505691.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_valid_split['Tariff_OK'] = np.where(X_valid_split['Tariff_OK'] == 'OK', 1, 0)


In [61]:
X_train_split.head()

,Gender,Age,Connect_Date,L_O_S,Dropped_Calls,tariff,Handset,Peak_calls_Sum,Peak_mins_Sum,OffPeak_calls_Sum,OffPeak_mins_Sum,Weekend_calls_Sum,Weekend_mins_Sum,International_mins_Sum,Nat_call_cost_Sum,AvePeak,AveOffPeak,AveWeekend,National_calls,National mins,AveNational,All_calls_mins,Dropped_calls_ratio,Usage_Band,Mins_charge,call_cost_per_min,actual call cost,Total_call_cost,Total_Cost,Tariff_OK,average cost min,Peak ratio,OffPeak ratio,Weekend ratio,Nat-InterNat Ratio,high Dropped calls,No Usage,id
0,F,50.0,870,29.200000,2.0,Play 100,BS210,62.0,153.000000,185.0,438.600001,4.0,29.000000,126.002615,2.045727,2.467742,2.370811,7.250000,251.0,620.600001,2.472510,746.602616,0.003984,Med,20.600001,9.930712,2.045727,52.446773,112.386773,1,0.150531,0.246536,0.706735,0.046729,0.203034,F,F,K262360
1,M,25.0,350,46.533333,1.0,CAT 100,ASAD90,146.0,718.800000,98.0,164.700000,4.0,37.200000,251.580636,41.072379,4.923288,1.680612,9.300000,248.0,920.700000,3.712500,1172.280636,0.002016,Med,320.700000,12.807103,41.072380,116.546571,221.546571,1,0.188988,0.780710,0.178886,0.040404,0.273249,F,F,K170160
3,F,59.0,924,27.400000,1.0,CAT 50,BS110,84.0,317.400001,57.0,161.699999,0.0,0.000000,23.998036,20.950771,3.778571,2.836842,0.000000,141.0,479.100000,3.397872,503.098036,0.003546,MedLow,179.100000,11.624922,20.820235,28.019646,111.419646,1,0.221467,0.662492,0.337508,0.000000,0.050090,F,F,K332460
4,F,25.0,1103,21.433333,1.0,Play 300,WC95,14.0,309.600000,326.0,637.800000,6.0,14.400000,87.051515,0.000000,22.114286,1.956442,2.400000,346.0,961.800000,2.779769,1048.851515,0.001445,Med,-838.200000,9.403618,0.000000,34.820606,112.760606,1,0.107509,0.321896,0.663132,0.014972,0.090509,F,F,K394220
5,M,27.0,579,38.900000,8.0,CAT 100,S50,170.0,414.600001,2.0,342.300000,21.0,38.399999,33.928464,19.946237,2.438824,171.150000,1.828571,193.0,795.300000,4.120725,829.228464,0.020725,Med,195.300000,10.213127,19.946237,30.124776,135.124776,1,0.162952,0.521313,0.430404,0.048284,0.042661,F,F,K286620


In [62]:
# Custom transformer to remove prefix from column names
class RemovePrefixTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, prefixes):
        self.prefixes = prefixes

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        for prefix in self.prefixes:
            X.columns = [col.split(f'{prefix}__')[1] if f'{prefix}__' in col else col for col in X.columns]
        return X

In [63]:
print(X_train_split['Usage_Band'].unique())

['Med' 'MedLow' 'MedHigh' 'Low' 'High' nan]


you can handle them explicitly before preprocessing, for example, by replacing them with the most frequent category using fillna()

In [64]:
X_train_split = X_train_split.copy()
X_train_split['Usage_Band'] = X_train_split['Usage_Band'].fillna(X_train_split['Usage_Band'].mode()[0])
print(X_train_split['Usage_Band'].unique())

# Handle missing values in 'Dropped_calls_ratio' by filling with the median
X_train_split['Dropped_calls_ratio'] = X_train_split['Dropped_calls_ratio'].fillna(X_train_split['Dropped_calls_ratio'].median())

# Handle missing values in 'call_cost_per_min' by filling with the median
X_train_split['call_cost_per_min'] = X_train_split['call_cost_per_min'].fillna(X_train_split['call_cost_per_min'].median())

['Med' 'MedLow' 'MedHigh' 'Low' 'High']


In [65]:
y_train_split = y_train_split.fillna(y_train_split.mode()[0])
y_valid_split = y_valid_split.fillna(y_train_split.mode()[0])

In [66]:
X_valid_split = X_valid_split.copy()

# Handle missing values in 'Dropped_calls_ratio' by filling with the median
X_valid_split['Dropped_calls_ratio'] = X_valid_split['Dropped_calls_ratio'].fillna(X_train_split['Dropped_calls_ratio'].median())

# Handle missing values in 'call_cost_per_min' by filling with the median
X_valid_split['call_cost_per_min'] = X_valid_split['call_cost_per_min'].fillna(X_train_split['call_cost_per_min'].median())

# Handle missing values in 'Usage_Band' for X_validation_split
X_valid_split['Usage_Band'] = X_valid_split['Usage_Band'].fillna(X_train_split['Usage_Band'].mode()[0])

In [67]:
X_test = X_test.copy()
X_test['Usage_Band'] = X_test['Usage_Band'].fillna(X_train_split['Usage_Band'].mode()[0])
print(X_test['Usage_Band'].unique())

# Handle missing values in 'Dropped_calls_ratio' by filling with the median
X_test['Dropped_calls_ratio'] = X_test['Dropped_calls_ratio'].fillna(X_train_split['Dropped_calls_ratio'].median())

# Handle missing values in 'call_cost_per_min' by filling with the median
X_test['call_cost_per_min'] = X_test['call_cost_per_min'].fillna(X_train_split['call_cost_per_min'].median())

['MedHigh' 'Med' 'High' 'MedLow' 'Low']


In [68]:
X_train_split.head()

,Gender,Age,Connect_Date,L_O_S,Dropped_Calls,tariff,Handset,Peak_calls_Sum,Peak_mins_Sum,OffPeak_calls_Sum,OffPeak_mins_Sum,Weekend_calls_Sum,Weekend_mins_Sum,International_mins_Sum,Nat_call_cost_Sum,AvePeak,AveOffPeak,AveWeekend,National_calls,National mins,AveNational,All_calls_mins,Dropped_calls_ratio,Usage_Band,Mins_charge,call_cost_per_min,actual call cost,Total_call_cost,Total_Cost,Tariff_OK,average cost min,Peak ratio,OffPeak ratio,Weekend ratio,Nat-InterNat Ratio,high Dropped calls,No Usage,id
0,F,50.0,870,29.200000,2.0,Play 100,BS210,62.0,153.000000,185.0,438.600001,4.0,29.000000,126.002615,2.045727,2.467742,2.370811,7.250000,251.0,620.600001,2.472510,746.602616,0.003984,Med,20.600001,9.930712,2.045727,52.446773,112.386773,1,0.150531,0.246536,0.706735,0.046729,0.203034,F,F,K262360
1,M,25.0,350,46.533333,1.0,CAT 100,ASAD90,146.0,718.800000,98.0,164.700000,4.0,37.200000,251.580636,41.072379,4.923288,1.680612,9.300000,248.0,920.700000,3.712500,1172.280636,0.002016,Med,320.700000,12.807103,41.072380,116.546571,221.546571,1,0.188988,0.780710,0.178886,0.040404,0.273249,F,F,K170160
3,F,59.0,924,27.400000,1.0,CAT 50,BS110,84.0,317.400001,57.0,161.699999,0.0,0.000000,23.998036,20.950771,3.778571,2.836842,0.000000,141.0,479.100000,3.397872,503.098036,0.003546,MedLow,179.100000,11.624922,20.820235,28.019646,111.419646,1,0.221467,0.662492,0.337508,0.000000,0.050090,F,F,K332460
4,F,25.0,1103,21.433333,1.0,Play 300,WC95,14.0,309.600000,326.0,637.800000,6.0,14.400000,87.051515,0.000000,22.114286,1.956442,2.400000,346.0,961.800000,2.779769,1048.851515,0.001445,Med,-838.200000,9.403618,0.000000,34.820606,112.760606,1,0.107509,0.321896,0.663132,0.014972,0.090509,F,F,K394220
5,M,27.0,579,38.900000,8.0,CAT 100,S50,170.0,414.600001,2.0,342.300000,21.0,38.399999,33.928464,19.946237,2.438824,171.150000,1.828571,193.0,795.300000,4.120725,829.228464,0.020725,Med,195.300000,10.213127,19.946237,30.124776,135.124776,1,0.162952,0.521313,0.430404,0.048284,0.042661,F,F,K286620


In [69]:
X_train_split.iloc[:, 25]

0        9.930712
1       12.807103
3       11.624922
4        9.403618
5       10.213127
          ...    
5038    13.427203
5039     8.079475
5040    10.557687
5042    11.464996
5043    10.533141
Name: call_cost_per_min, Length: 4036, dtype: float64

In [127]:
# Define columns to drop
columns_to_drop = ['id']  # Drop because it's not numerical, later on add it back to know which prediction corresponds to which individual

# Define columns for different encoding methods
one_hot_encode_columns = ['Gender', 'high Dropped calls', 'No Usage']
woe_encode_columns = ['tariff', 'Handset', 'Usage_Band'] #ipv ordinal endoding

preprocessor = ColumnTransformer(
    transformers=[
        ('drop_columns', 'drop', columns_to_drop),
        ('one_hot_encode', OneHotEncoder(drop='first', sparse_output=False), one_hot_encode_columns),
        ('WOE_encode', WOEEncoder(), woe_encode_columns),
    ],
    remainder='passthrough'  # Keep the remaining columns as they are
)

# Build the preprocessing pipeline
preprocessing_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('remove_prefix', RemovePrefixTransformer(prefixes=['one_hot_encode', 'WOE_encode', 'remainder']))   # Add this step to remove the prefix
])

#lgb_classifier = lgb.LGBMClassifier(is_unbalance=True)
lgb_classifier = lgb.LGBMClassifier(scale_pos_weight=(1 - y_train_split.sum() / len(y_train_split)))

lgb_pipeline = Pipeline(steps=[
    ('preprocessing', preprocessing_pipeline),
    ('model', lgb_classifier)
])

In [128]:
# Function to calculate profit metric on test set
def calculate_profit_metric(y_probabilities, dataset):
    # Extract probabilities for positive class
    churn_probabilities = y_probabilities[:, 1]

    # Create DataFrame with churn probabilities and corresponding profitability
    profit_df = pd.DataFrame({"churn_prob": churn_probabilities, "profit": dataset["average cost min"]})

    # Sort customers by predicted probabilities in descending order
    profit_df = profit_df.sort_values(by='churn_prob', ascending=False)

    # Calculate profit @ top-20
    top_20_profit = profit_df["profit"][:20].sum()

    return top_20_profit

def calculate_profit_metric_2(y_true, y_probabilities, dataset, top_k=20):
    # Sort customers by predicted probabilities in descending order
    sorted_indices = sorted(range(len(y_probabilities)), key=lambda k: y_probabilities[k, 1], reverse=True)

    # Identify the top-k churners
    top_k_indices = sorted_indices[:top_k]

    # Calculate profit at top-k churners
    profit = sum(dataset.iloc[i]["average cost min"] for i in top_k_indices if y_true[i] == 1)

    return profit

'''
# Define evaluation metrics
def profit_at_top_20(y_true, y_pred, top_k=20):
    # Sort customers by predicted probabilities in descending order
    sorted_indices = sorted(range(len(y_pred)), key=lambda k: y_pred[k, 1], reverse=True)

    # Identify the top-20 customers
    top_20_indices = sorted_indices[:top_k]

    # Calculate profit at top-20
    profit = sum(y_true[i] * y_pred[i, 1] for i in top_20_indices)

    return profit

profit_at_top_20_scorer = make_scorer(profit_at_top_20, greater_is_better=True)
'''

def profit_at_top_20(y_true, y_pred, dataset, top_k=20):
    # Sort customers by predicted probabilities in descending order
    sorted_indices = sorted(range(len(y_pred)), key=lambda k: y_pred[k, 1], reverse=True)

    # Identify the top-k churners
    top_k_indices = sorted_indices[:top_k]

    # Calculate profit at top-k churners
    profit = sum(dataset.iloc[i]["average cost min"] for i in top_k_indices if y_true[i] == 1)

    return profit

# Define custom scorer
profit_at_top_20_scorer = make_scorer(profit_at_top_20, greater_is_better=True, needs_proba=True)

In [129]:
# Assuming X_train_split is your training data
X_train_preprocessed = preprocessing_pipeline.fit_transform(X_train_split, y_train_split)
X_train_preprocessed

,Gender_M,high Dropped calls_T,No Usage_T,tariff,Handset,Usage_Band,Age,Connect_Date,L_O_S,Dropped_Calls,Peak_calls_Sum,Peak_mins_Sum,OffPeak_calls_Sum,OffPeak_mins_Sum,Weekend_calls_Sum,Weekend_mins_Sum,International_mins_Sum,Nat_call_cost_Sum,AvePeak,AveOffPeak,AveWeekend,National_calls,National mins,AveNational,All_calls_mins,Dropped_calls_ratio,Mins_charge,call_cost_per_min,actual call cost,Total_call_cost,Total_Cost,Tariff_OK,average cost min,Peak ratio,OffPeak ratio,Weekend ratio,Nat-InterNat Ratio
0,0.0,0.0,0.0,0.420413,-1.012710,-0.242724,50.0,870,29.200000,2.0,62.0,153.000000,185.0,438.600001,4.0,29.000000,126.002615,2.045727,2.467742,2.370811,7.250000,251.0,620.600001,2.472510,746.602616,0.003984,20.600001,9.930712,2.045727,52.446773,112.386773,1,0.150531,0.246536,0.706735,0.046729,0.203034
1,1.0,0.0,0.0,-0.137133,2.964342,-0.242724,25.0,350,46.533333,1.0,146.0,718.800000,98.0,164.700000,4.0,37.200000,251.580636,41.072379,4.923288,1.680612,9.300000,248.0,920.700000,3.712500,1172.280636,0.002016,320.700000,12.807103,41.072380,116.546571,221.546571,1,0.188988,0.780710,0.178886,0.040404,0.273249
3,0.0,0.0,0.0,-0.041542,0.026744,0.760057,59.0,924,27.400000,1.0,84.0,317.400001,57.0,161.699999,0.0,0.000000,23.998036,20.950771,3.778571,2.836842,0.000000,141.0,479.100000,3.397872,503.098036,0.003546,179.100000,11.624922,20.820235,28.019646,111.419646,1,0.221467,0.662492,0.337508,0.000000,0.050090
4,0.0,0.0,0.0,-0.504292,-2.631809,-0.242724,25.0,1103,21.433333,1.0,14.0,309.600000,326.0,637.800000,6.0,14.400000,87.051515,0.000000,22.114286,1.956442,2.400000,346.0,961.800000,2.779769,1048.851515,0.001445,-838.200000,9.403618,0.000000,34.820606,112.760606,1,0.107509,0.321896,0.663132,0.014972,0.090509
5,1.0,0.0,0.0,-0.137133,-0.111923,-0.242724,27.0,579,38.900000,8.0,170.0,414.600001,2.0,342.300000,21.0,38.399999,33.928464,19.946237,2.438824,171.150000,1.828571,193.0,795.300000,4.120725,829.228464,0.020725,195.300000,10.213127,19.946237,30.124776,135.124776,1,0.162952,0.521313,0.430404,0.048284,0.042661
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5038,0.0,0.0,0.0,-0.137133,0.026744,-0.242724,22.0,1270,15.866667,1.0,26.0,721.200000,14.0,75.600001,29.0,59.000000,104.796650,34.346783,27.738462,5.400000,2.034483,69.0,855.800001,12.402899,960.596651,0.007246,255.800001,13.427203,34.346784,65.785779,170.785779,1,0.177791,0.842720,0.088338,0.068941,0.122455
5039,0.0,0.0,0.0,0.024608,2.964342,-0.021064,16.0,623,37.433333,2.0,151.0,1169.400001,201.0,657.900000,34.0,71.400001,242.983418,56.451295,7.744371,3.273134,2.100000,386.0,1898.700002,4.918912,2141.683420,0.002591,698.700002,8.079475,56.451295,129.346320,279.346320,1,0.130433,0.615895,0.346500,0.037605,0.127974
5040,0.0,0.0,0.0,-0.137133,2.984962,-0.242724,29.0,271,49.166667,1.0,135.0,405.600000,124.0,301.200001,11.0,23.000000,174.140881,13.703878,3.004444,2.429032,2.090909,270.0,729.800001,2.702963,903.940882,0.001852,129.800001,10.557687,13.703878,65.946142,170.946142,1,0.189112,0.555769,0.412716,0.031515,0.238615
5042,1.0,0.0,0.0,0.420413,-0.111923,0.760057,46.0,790,31.866667,2.0,72.0,112.200000,31.0,230.700000,2.0,4.200000,59.510484,0.000000,1.558333,7.441935,2.100000,105.0,347.100000,3.305714,406.610484,0.009524,-252.900000,11.464996,0.000000,23.804194,83.744194,1,0.205957,0.323250,0.664650,0.012100,0.171451


In [130]:
unique_dtypes = X_train_preprocessed.dtypes.unique()
print("Unique data types:")
print(unique_dtypes)

Unique data types:
[dtype('float64') dtype('int64') dtype('int32')]


In [131]:
# Show rows with missing values in the 'Usage_Band' column
missing_values = X_train_preprocessed[X_train_preprocessed.isnull().any(axis=1)]
missing_values

,Gender_M,high Dropped calls_T,No Usage_T,tariff,Handset,Usage_Band,Age,Connect_Date,L_O_S,Dropped_Calls,Peak_calls_Sum,Peak_mins_Sum,OffPeak_calls_Sum,OffPeak_mins_Sum,Weekend_calls_Sum,Weekend_mins_Sum,International_mins_Sum,Nat_call_cost_Sum,AvePeak,AveOffPeak,AveWeekend,National_calls,National mins,AveNational,All_calls_mins,Dropped_calls_ratio,Mins_charge,call_cost_per_min,actual call cost,Total_call_cost,Total_Cost,Tariff_OK,average cost min,Peak ratio,OffPeak ratio,Weekend ratio,Nat-InterNat Ratio


#### LGB

In [141]:
# Create parameter grid for LightGBM hyperparameter tuning
lgb_param_grid = {
    'n_estimators': [140, 160, 180, 200],
    'max_depth': [1, 3, 5, 7],
    'learning_rate': [0.01, 0.03, 0.05, 0.1,],
}

X_train_preprocessed = preprocessing_pipeline.fit_transform(X_train_split, y_train_split)

# Create GridSearchCV instance for LightGBM
lgb_grid_search = GridSearchCV(lgb_classifier, lgb_param_grid, scoring={'profit_at_top_20': profit_at_top_20_scorer}, refit='profit_at_top_20', verbose=0, cv=5, n_jobs=-1)

# Fit the GridSearchCV on training data for LightGBM
lgb_grid_search.fit(X_train_preprocessed, y_train_split)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 596, number of negative: 3440
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002784 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7077
[LightGBM] [Info] Number of data points in the train set: 4036, number of used features: 36
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.147671 -> initscore=-1.752986
[LightGBM] [Info] Start training from score -1.752986


c:\Users\lenne\anaconda3\envs\AA\lib\site-packages\sklearn\model_selection\_search.py:976: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


GridSearchCV(cv=5,
             estimator=LGBMClassifier(scale_pos_weight=0.8523290386521308),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.03, 0.05, 0.1],
                         'max_depth': [1, 3, 5, 7],
                         'n_estimators': [140, 160, 180, 200]},
             refit='profit_at_top_20',
             scoring={'profit_at_top_20': make_scorer(profit_at_top_20, needs_proba=True)})

In [142]:
lgb_grid_search.best_estimator_ #has the optimal hyperparameters

LGBMClassifier(learning_rate=0.01, max_depth=1, n_estimators=140,
               scale_pos_weight=0.8523290386521308)

In [143]:
print("Best Score:", lgb_grid_search.best_score_)
print("Best Parameters:", lgb_grid_search.best_params_)

Best Score: nan
Best Parameters: {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 140}


In [144]:
# Get the best LightGBM model from the grid search
best_lgb_model = lgb_grid_search.best_estimator_

# Fit the best model on the training data
best_lgb_model.fit(X_train_preprocessed, y_train_split) 

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 596, number of negative: 3440
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002125 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7077
[LightGBM] [Info] Number of data points in the train set: 4036, number of used features: 36
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.147671 -> initscore=-1.752986
[LightGBM] [Info] Start training from score -1.752986


LGBMClassifier(learning_rate=0.01, max_depth=1, n_estimators=140,
               scale_pos_weight=0.8523290386521308)

In [145]:
# Apply preprocessing pipeline to the validation set IPV FIT_TRANSFORM GWN TRANSFORM BC INFO VAN TRAINING SET
X_valid_preprocessed = preprocessing_pipeline.transform(X_valid_split)

In [146]:
pred = best_lgb_model.predict(X_valid_preprocessed)

In [147]:
# Evaluate on the validation set for LightGBM
y_valid_probabilities_lgb = best_lgb_model.predict_proba(X_valid_preprocessed)

In [148]:
def calculate_profit_metric_2(y_true, y_probabilities, dataset, top_k=20):
    # Convert y_true to a NumPy array to align with y_probabilities
    y_true = y_true.values.flatten()

    # Sort customers by predicted probabilities in descending order
    sorted_indices = sorted(range(len(y_probabilities)), key=lambda k: y_probabilities[k, 1], reverse=True)

    # Identify the top-k churners
    top_k_indices = sorted_indices[:top_k]

    # Calculate profit at top-k churners
    profit = sum(dataset.iloc[i]["average cost min"] for i in top_k_indices if y_true[i] == 1)

    return profit

In [149]:
# Evaluate on the validation set for LightGBM
auc_score_lgb = roc_auc_score(y_valid_split, y_valid_probabilities_lgb[:, 1])

# Calculate profit metric
profit_metric = calculate_profit_metric(y_valid_probabilities_lgb, X_valid_split)

profit_metric2 = calculate_profit_metric_2(y_valid_split, y_valid_probabilities_lgb, X_valid_split, top_k=20)

print(f'AUC for LightGBM on Validation Set: {auc_score_lgb}')
print(f'Profit @ Top-20 for LightGBM on Validation Set: {profit_metric}')
print(f'Profit @ Top-20 for LightGBM on Validation Set (I THOUGHT THIS IS MORE CORRECT BUT SEEMS NOT): {profit_metric2}')

AUC for LightGBM on Validation Set: 0.8985123954028018
Profit @ Top-20 for LightGBM on Validation Set: 3.6501200000000003
Profit @ Top-20 for LightGBM on Validation Set (I THOUGHT THIS IS MORE CORRECT BUT SEEMS NOT): 3.0037110000000005


In [150]:
# Access the best hyperparameters for LightGBM
best_hyperparameters_LGB = lgb_grid_search.best_params_
print(f'Best Hyperparameters for LightGBM: {best_hyperparameters_LGB}')

Best Hyperparameters for LightGBM: {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 140}


In [151]:
# Apply preprocessing pipeline to the validation set
X_test_preprocessed = preprocessing_pipeline.transform(X_test)

In [152]:
# Evaluate on the test set for LightGBM
y_test_probabilities_lgb = best_lgb_model.predict_proba(X_test_preprocessed)
'''
auc_score_test_lgb = roc_auc_score(y_test, y_test_probabilities_lgb[:, 1])
print(f'AUC for LightGBM on Test Set: {auc_score_test_lgb}')
'''

"\nauc_score_test_lgb = roc_auc_score(y_test, y_test_probabilities_lgb[:, 1])\nprint(f'AUC for LightGBM on Test Set: {auc_score_test_lgb}')\n"

In [153]:
y_test_probabilities_lgb = pd.DataFrame(y_test_probabilities_lgb, columns=['PROB_0', 'PROB_1'])
y_test_probabilities_lgb_with_id = pd.concat([data_test['id'], y_test_probabilities_lgb], axis=1)
y_test_probabilities_lgb_with_id

,id,PROB_0,PROB_1
0,K751808,0.929461,0.070539
1,K837351,0.879197,0.120803
2,K548114,0.929461,0.070539
3,K736156,0.929461,0.070539
4,K508080,0.929461,0.070539
...,...,...,...
1677,K588314,0.879197,0.120803
1678,K826807,0.879197,0.120803
1679,K982731,0.879197,0.120803
1680,K623037,0.879197,0.120803


In [154]:
result_LGB = y_test_probabilities_lgb_with_id.iloc[:, [0, 2]]
result_LGB.to_csv('result_LGB_6.csv', header=False, index=False)

In [155]:
result_LGB

,id,PROB_1
0,K751808,0.070539
1,K837351,0.120803
2,K548114,0.070539
3,K736156,0.070539
4,K508080,0.070539
...,...,...
1677,K588314,0.120803
1678,K826807,0.120803
1679,K982731,0.120803
1680,K623037,0.120803


### Option 2

In Option 1, you're directly tuning the hyperparameters of the LightGBM classifier (lgb_classifier) without considering the preprocessing steps. This means the hyperparameters are optimized based solely on the transformed features. LightGBM is giving you a warning about the parameters because it's detecting potential issues with the parameter values, but it still proceeds with the training.

In Option 2, you're using the entire pipeline (lgb_pipeline) within the GridSearchCV. This includes the preprocessing steps and the LightGBM model. **The warnings you see are related to the features being preprocessed, and they do not affect the model optimization process. However, the optimization process may be affected by the preprocessing steps if they are not handled properly.**

The difference in results, particularly the warning about non-finite test scores and the change in the best parameters and score, may indicate that the preprocessing steps in Option 2 are affecting the optimization process. It's possible that the preprocessing steps are causing issues such as feature scaling problems or missing values, leading to non-finite test scores and affecting the optimization process.

In [157]:
# Create parameter grid for LightGBM hyperparameter tuning
lgb_param_grid = {
    'model__n_estimators': [140, 160, 180, 200],
    'model__max_depth': [1, 3, 5, 7],
    'model__learning_rate': [0.01, 0.03, 0.05, 0.1,],
}

# Create GridSearchCV instance for LightGBM
lgb_grid_search = GridSearchCV(lgb_pipeline, lgb_param_grid, scoring={'profit_at_top_20': profit_at_top_20_scorer}, refit='profit_at_top_20', verbose=0, cv=5, n_jobs=-1)

# Fit the GridSearchCV on training data for LightGBM
lgb_grid_search.fit(X_train_split, y_train_split)

c:\Users\lenne\anaconda3\envs\AA\lib\site-packages\sklearn\model_selection\_search.py:976: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 596, number of negative: 3440
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001254 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 7077
[LightGBM] [Info] Number of data points in the train set: 4036, number of used features: 36
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.147671 -> initscore=-1.752986
[LightGBM] [Info] Start training from score -1.752986


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing',
                                        Pipeline(steps=[('preprocessor',
                                                         ColumnTransformer(remainder='passthrough',
                                                                           transformers=[('drop_columns',
                                                                                          'drop',
                                                                                          ['id']),
                                                                                         ('one_hot_encode',
                                                                                          OneHotEncoder(drop='first',
                                                                                                        sparse_output=False),
                                                                                          ['Gender',
                                                                                           'high '
                                                                                           'Dropped '
                                                                                           'calls',
                                                                                           'No '
                                                                                           'Usage']),
                                                                                         ('WOE_encode',
                                                                                          WOEEncoder(),
                                                                                          ['tariff',
                                                                                           'Handset',
                                                                                           'Usage_Ban...
                                                         RemovePrefixTransformer(prefixes=['one_hot_encode',
                                                                                           'WOE_encode',
                                                                                           'remainder']))])),
                                       ('model',
                                        LGBMClassifier(scale_pos_weight=0.8523290386521308))]),
             n_jobs=-1,
             param_grid={'model__learning_rate': [0.01, 0.03, 0.05, 0.1],
                         'model__max_depth': [1, 3, 5, 7],
                         'model__n_estimators': [140, 160, 180, 200]},
             refit='profit_at_top_20',
             scoring={'profit_at_top_20': make_scorer(profit_at_top_20, needs_proba=True)})

In [158]:
lgb_grid_search.best_estimator_ #has the optimal hyperparameters

Pipeline(steps=[('preprocessing',
                 Pipeline(steps=[('preprocessor',
                                  ColumnTransformer(remainder='passthrough',
                                                    transformers=[('drop_columns',
                                                                   'drop',
                                                                   ['id']),
                                                                  ('one_hot_encode',
                                                                   OneHotEncoder(drop='first',
                                                                                 sparse_output=False),
                                                                   ['Gender',
                                                                    'high '
                                                                    'Dropped '
                                                                    'calls',
                                                                    'No '
                                                                    'Usage']),
                                                                  ('WOE_encode',
                                                                   WOEEncoder(),
                                                                   ['tariff',
                                                                    'Handset',
                                                                    'Usage_Band'])])),
                                 ('remove_prefix',
                                  RemovePrefixTransformer(prefixes=['one_hot_encode',
                                                                    'WOE_encode',
                                                                    'remainder']))])),
                ('model',
                 LGBMClassifier(learning_rate=0.01, max_depth=1,
                                n_estimators=140,
                                scale_pos_weight=0.8523290386521308))])

In [160]:
print("Best Score:", lgb_grid_search.best_score_)
print("Best Parameters:", lgb_grid_search.best_params_)

# Get the best LightGBM model from the grid search
best_lgb_model = lgb_grid_search.best_estimator_

# Fit the best model on the training data
best_lgb_model.fit(X_train_split, y_train_split) 

Best Score: nan
Best Parameters: {'model__learning_rate': 0.01, 'model__max_depth': 1, 'model__n_estimators': 140}
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 596, number of negative: 3440
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001843 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7077
[LightGBM] [Info] Number of data points in the train set: 4036, number of used features: 36
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.147671 -> initscore=-1.752986
[LightGBM] [Info] Start training from score -1.752986


Pipeline(steps=[('preprocessing',
                 Pipeline(steps=[('preprocessor',
                                  ColumnTransformer(remainder='passthrough',
                                                    transformers=[('drop_columns',
                                                                   'drop',
                                                                   ['id']),
                                                                  ('one_hot_encode',
                                                                   OneHotEncoder(drop='first',
                                                                                 sparse_output=False),
                                                                   ['Gender',
                                                                    'high '
                                                                    'Dropped '
                                                                    'calls',
                                                                    'No '
                                                                    'Usage']),
                                                                  ('WOE_encode',
                                                                   WOEEncoder(),
                                                                   ['tariff',
                                                                    'Handset',
                                                                    'Usage_Band'])])),
                                 ('remove_prefix',
                                  RemovePrefixTransformer(prefixes=['one_hot_encode',
                                                                    'WOE_encode',
                                                                    'remainder']))])),
                ('model',
                 LGBMClassifier(learning_rate=0.01, max_depth=1,
                                n_estimators=140,
                                scale_pos_weight=0.8523290386521308))])

In [162]:
# Evaluate on the validation set for LightGBM
y_valid_probabilities_lgb = best_lgb_model.predict_proba(X_valid_split)

def calculate_profit_metric_2(y_true, y_probabilities, dataset, top_k=20):
    # Convert y_true to a NumPy array to align with y_probabilities
    y_true = y_true.values.flatten()

    # Sort customers by predicted probabilities in descending order
    sorted_indices = sorted(range(len(y_probabilities)), key=lambda k: y_probabilities[k, 1], reverse=True)

    # Identify the top-k churners
    top_k_indices = sorted_indices[:top_k]

    # Calculate profit at top-k churners
    profit = sum(dataset.iloc[i]["average cost min"] for i in top_k_indices if y_true[i] == 1)

    return profit

# Evaluate on the validation set for LightGBM
auc_score_lgb = roc_auc_score(y_valid_split, y_valid_probabilities_lgb[:, 1])

# Calculate profit metric
profit_metric = calculate_profit_metric(y_valid_probabilities_lgb, X_valid_split)

profit_metric2 = calculate_profit_metric_2(y_valid_split, y_valid_probabilities_lgb, X_valid_split, top_k=20)

print(f'AUC for LightGBM on Validation Set: {auc_score_lgb}')
print(f'Profit @ Top-20 for LightGBM on Validation Set: {profit_metric}')
print(f'Profit @ Top-20 for LightGBM on Validation Set (I THOUGHT THIS IS MORE CORRECT BUT SEEMS NOT): {profit_metric2}')

AUC for LightGBM on Validation Set: 0.8985123954028018
Profit @ Top-20 for LightGBM on Validation Set: 3.6501200000000003
Profit @ Top-20 for LightGBM on Validation Set (I THOUGHT THIS IS MORE CORRECT BUT SEEMS NOT): 3.0037110000000005


### Other approach

In [166]:
lgb_classifier = lgb.LGBMClassifier(is_unbalance=True)

lgb_pipeline = Pipeline(steps=[
    ('preprocessing', preprocessing_pipeline),
    ('model', lgb_classifier)
])

# Define evaluation metrics
def profit_at_top_20(y_true, y_pred, top_k=20):
    # Sort customers by predicted probabilities in descending order
    sorted_indices = sorted(range(len(y_pred)), key=lambda k: y_pred[k, 1], reverse=True)

    # Identify the top-20 customers
    top_20_indices = sorted_indices[:top_k]

    # Calculate profit at top-20
    profit = sum(y_true[i] * y_pred[i, 1] for i in top_20_indices)

    return profit

profit_at_top_20_scorer = make_scorer(profit_at_top_20, greater_is_better=True)

In [167]:
# Create parameter grid for LightGBM hyperparameter tuning
lgb_param_grid = {
    'model__n_estimators': [150, 160, 170, 180, 190, 200],
    'model__max_depth': [1, 2, 3, 4, 5],
    'model__learning_rate': [0.01, 0.05, 0.1, 0.2],
}

# Create GridSearchCV instance for LightGBM
lgb_grid_search = GridSearchCV(lgb_pipeline, lgb_param_grid, scoring={'profit_at_top_20': profit_at_top_20_scorer, 'auc': 'roc_auc'}, refit='profit_at_top_20', verbose=0, cv=5, n_jobs=-1)


# Fit the GridSearchCV on training data for LightGBM
lgb_grid_search.fit(X_train_split, y_train_split)

c:\Users\lenne\anaconda3\envs\AA\lib\site-packages\sklearn\model_selection\_search.py:976: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 596, number of negative: 3440
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002722 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 7077
[LightGBM] [Info] Number of data points in the train set: 4036, number of used features: 36
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.147671 -> initscore=-1.752986
[LightGBM] [Info] Start training from score -1.752986


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing',
                                        Pipeline(steps=[('preprocessor',
                                                         ColumnTransformer(remainder='passthrough',
                                                                           transformers=[('drop_columns',
                                                                                          'drop',
                                                                                          ['id']),
                                                                                         ('one_hot_encode',
                                                                                          OneHotEncoder(drop='first',
                                                                                                        sparse_output=False),
                                                                                          ['Gender',
                                                                                           'high '
                                                                                           'Dropped '
                                                                                           'calls',
                                                                                           'No '
                                                                                           'Usage']),
                                                                                         ('WOE_encode',
                                                                                          WOEEncoder(),
                                                                                          ['tariff',
                                                                                           'Handset',
                                                                                           'Usage_Ban...
                                                         RemovePrefixTransformer(prefixes=['one_hot_encode',
                                                                                           'WOE_encode',
                                                                                           'remainder']))])),
                                       ('model',
                                        LGBMClassifier(is_unbalance=True))]),
             n_jobs=-1,
             param_grid={'model__learning_rate': [0.01, 0.05, 0.1, 0.2],
                         'model__max_depth': [1, 2, 3, 4, 5],
                         'model__n_estimators': [150, 160, 170, 180, 190, 200]},
             refit='profit_at_top_20',
             scoring={'auc': 'roc_auc',
                      'profit_at_top_20': make_scorer(profit_at_top_20)})

In [168]:
# Get the best LightGBM model from the grid search
best_lgb_model = lgb_grid_search.best_estimator_

# Fit the best model on the training data
best_lgb_model.fit(X_train_split, y_train_split) 

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 596, number of negative: 3440
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009399 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 7077
[LightGBM] [Info] Number of data points in the train set: 4036, number of used features: 36
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.147671 -> initscore=-1.752986
[LightGBM] [Info] Start training from score -1.752986


Pipeline(steps=[('preprocessing',
                 Pipeline(steps=[('preprocessor',
                                  ColumnTransformer(remainder='passthrough',
                                                    transformers=[('drop_columns',
                                                                   'drop',
                                                                   ['id']),
                                                                  ('one_hot_encode',
                                                                   OneHotEncoder(drop='first',
                                                                                 sparse_output=False),
                                                                   ['Gender',
                                                                    'high '
                                                                    'Dropped '
                                                                    'calls',
                                                                    'No '
                                                                    'Usage']),
                                                                  ('WOE_encode',
                                                                   WOEEncoder(),
                                                                   ['tariff',
                                                                    'Handset',
                                                                    'Usage_Band'])])),
                                 ('remove_prefix',
                                  RemovePrefixTransformer(prefixes=['one_hot_encode',
                                                                    'WOE_encode',
                                                                    'remainder']))])),
                ('model',
                 LGBMClassifier(is_unbalance=True, learning_rate=0.01,
                                max_depth=1, n_estimators=150))])

In [169]:
# Evaluate on the validation set
y_valid_probabilities_lgb = best_lgb_model.predict_proba(X_valid_split)

# Evaluate on the validation set for LightGBM
auc_score_lgb = roc_auc_score(y_valid_split, y_valid_probabilities_lgb[:, 1])

# Calculate profit metric
profit_metric = calculate_profit_metric(y_valid_probabilities_lgb, X_valid_split)

profit_metric2 = calculate_profit_metric_2(y_valid_split, y_valid_probabilities_lgb, X_valid_split, top_k=20)

print(f'AUC for LightGBM on Validation Set: {auc_score_lgb}')
print(f'Profit @ Top-20 for LightGBM on Validation Set: {profit_metric}')
print(f'Profit @ Top-20 for LightGBM on Validation Set (I THOUGHT THIS IS MORE CORRECT BUT SEEMS NOT): {profit_metric2}')

AUC for LightGBM on Validation Set: 0.8868162605183176
Profit @ Top-20 for LightGBM on Validation Set: 4.134671
Profit @ Top-20 for LightGBM on Validation Set (I THOUGHT THIS IS MORE CORRECT BUT SEEMS NOT): 2.1465739999999998


In [170]:
# Evaluate on the test set for LightGBM
y_test_probabilities_lgb = best_lgb_model.predict_proba(X_test)
y_test_probabilities_lgb = pd.DataFrame(y_test_probabilities_lgb, columns=['PROB_0', 'PROB_1'])
y_test_probabilities_lgb_with_id = pd.concat([data_test['id'], y_test_probabilities_lgb], axis=1)
y_test_probabilities_lgb_with_id

,id,PROB_0,PROB_1
0,K751808,0.819790,0.180210
1,K837351,0.604019,0.395981
2,K548114,0.819790,0.180210
3,K736156,0.819790,0.180210
4,K508080,0.819790,0.180210
...,...,...,...
1677,K588314,0.604019,0.395981
1678,K826807,0.604019,0.395981
1679,K982731,0.604019,0.395981
1680,K623037,0.604019,0.395981


In [171]:
result_LGB = y_test_probabilities_lgb_with_id.iloc[:, [0, 2]]
result_LGB.to_csv('result_LGB_7.csv', header=False, index=False)

In [ ]:
# SAME AS RESULT 1! my code says higher profit for validation 4.13 ipv 3.65 maar op scorebord gaat het achteruit: van 3.97 naar 3.81